# Hyperparameter tuning

## Dask cluster

<img src="../_img/dask-horizontal.svg" width="400">

In [ ]:
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'PULocationID', 
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'tip_fraction'

# Initialize Dask Cluster

In [ ]:
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers, 
    scheduler_size='medium',
    worker_size='large', 
    nthreads=2,
)
client = Client(cluster)
cluster

Open the dashboard (link above ^) and watch it when you execute some commands, you'll see which tasks are running across the cluster.

If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster from the "Dask" page in Saturn if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

# Load data and feature engineering

Load a sample from a single month for this exercise. Note we are loading the data with Dask now (`dd.read_csv` vs. `pd.read_csv`)

In [ ]:
import numpy as np
import dask.dataframe as dd

taxi = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv',
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    storage_options={'anon': True},
    assume_missing=True,
).sample(frac=0.01, replace=False)

In [ ]:
print(f'Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum().compute() / 1e6} MB')

In [ ]:
def prep_df(df: dd.DataFrame) -> dd.DataFrame:
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
    
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.isocalendar().week
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [y_col]].astype(float).fillna(-1)
    
    return df
    
taxi_train = prep_df(taxi)

In [ ]:
taxi_train.head()

# Run grid search

We use the preprocessing and `GridSearchCV` classes from dask-ml, but still use the scikit-learn `ElasticNet` model.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from dask_ml.compose import ColumnTransformer
from dask_ml.preprocessing import StandardScaler, DummyEncoder, Categorizer
from dask_ml.model_selection import GridSearchCV

pipeline = Pipeline(steps=[
    ('categorize', Categorizer(columns=categorical_feat)),
    ('onehot', DummyEncoder(columns=categorical_feat)),
    ('scale', ColumnTransformer(
        transformers=[('num', StandardScaler(), numeric_feat)], 
        remainder='passthrough',
    )),
    ('clf', ElasticNet(normalize=False, max_iter=100)),
])

params = {
    'clf__l1_ratio': np.arange(0, 1.1, 0.1),
    'clf__alpha': [0, 0.5, 1, 2],
}

grid_search = GridSearchCV(pipeline, params, cv=3, scoring='neg_mean_squared_error')

Open up the Dask dashboard after you run the cell below, you'll see the grid search in action!

In [ ]:
%%time
_ = grid_search.fit(taxi_train[features], taxi_train[y_col])
grid_search.best_score_

In [ ]:
grid_search.best_params_

## Save model

`GridSearchCV` automatically fits the best parameters to the full data and stores in `best_estimator_`

In [ ]:
import cloudpickle
import os

MODEL_PATH = 'models'
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f'{MODEL_PATH}/elastic_net_dask.pkl', 'wb') as f:
    cloudpickle.dump(grid_search.best_estimator_, f)

## Calculate metrics on test set

Use a different month for test set

In [ ]:
taxi_test = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv',
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    storage_options={'anon': True},
    assume_missing=True,
).sample(frac=0.01, replace=False)

taxi_test = prep_df(taxi_test)

In [ ]:
from sklearn.metrics import mean_squared_error

preds = grid_search.predict(taxi_test[features])
mean_squared_error(taxi_test[y_col], preds, squared=False)